## CNN Based Autoencoder

Source: https://www.geeksforgeeks.org/implement-convolutional-autoencoder-in-pytorch-with-cuda/



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
from PIL import Image

# Define the autoencoder architecture
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16,
                               kernel_size=3,
                               stride=2,
                               padding=1,
                               output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3,
                               kernel_size=3,
                               stride=2,
                               padding=1,
                               output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Define PSNR calculation function
def calculate_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    max_pixel_value = 1.0  # Assuming images are normalized between 0 and 1

    psnr = 10 * np.log10((max_pixel_value ** 2) / mse)
    return psnr

# Define transform
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train an autoencoder for each folder
for folder_name in ['25', '50', '75']:
    # Load dataset from 'noisy_train' and 'noisy_test' folders
    train_dataset = datasets.ImageFolder(root=f'/content/drive/MyDrive/Colab Notebooks/pneumonia/noisy_train/{folder_name}', transform=transform)
    test_dataset = datasets.ImageFolder(root=f'/content/drive/MyDrive/Colab Notebooks/pneumonia/noisy_test/{folder_name}', transform=transform)

    # Define the dataloader
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=128)

    # Initialize the autoencoder
    model = Autoencoder().to(device)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the autoencoder
    num_epochs = 50
    for epoch in range(num_epochs):
        for data in train_loader:
            img, _ = data
            img = img.to(device)
            optimizer.zero_grad()
            output = model(img)
            loss = criterion(output, img)
            loss.backward()
            optimizer.step()
        if epoch % 5 == 0:
            print(f'Folder {folder_name}, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Evaluate PSNR on the test dataset
    psnr_total = 0.0
    num_samples = 0

    for i, (img, paths) in enumerate(test_loader):
        img = img.to(device)
        output = model(img)

        # Iterate over batch images
        for i in range(img.size(0)):
            index = num_samples + i
            if index < len(test_dataset.samples):  # Check if the index is within the range
                original_img_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/pneumonia/test', os.path.basename(test_dataset.samples[index][0]))
                original_img = np.array(Image.open(original_img_path).convert('RGB')) / 255.0  # Normalize to [0, 1]
                original_img_tensor = torch.tensor(original_img).permute(2, 0, 1).unsqueeze(0).float().to(device)

                # Resize the output to match the original image size
                resized_output = torch.nn.functional.interpolate(output[i].unsqueeze(0), size=original_img_tensor.shape[-2:], mode='bilinear', align_corners=False)

            # Calculate PSNR
            psnr = calculate_psnr(original_img_tensor.cpu().numpy(), resized_output.detach().cpu().numpy())
            psnr_total += psnr
            num_samples += 1

    average_psnr = psnr_total / num_samples
    print('Average PSNR on test dataset: {:.4f} dB'.format(average_psnr))

    # Save the model
    torch.save(model.state_dict(), f'conv_autoencoder_{folder_name}.pth')

Folder 25, Epoch [1/50], Loss: 0.0846
Folder 25, Epoch [6/50], Loss: 0.0762
Folder 25, Epoch [11/50], Loss: 0.0643
Folder 25, Epoch [16/50], Loss: 0.0622
Folder 25, Epoch [21/50], Loss: 0.0519
Folder 25, Epoch [26/50], Loss: 0.0468
Folder 25, Epoch [31/50], Loss: 0.0460
Folder 25, Epoch [36/50], Loss: 0.0367
Folder 25, Epoch [41/50], Loss: 0.0290
Folder 25, Epoch [46/50], Loss: 0.0235
Average PSNR on test dataset: 13.2916 dB
Folder 50, Epoch [1/50], Loss: 0.0900
Folder 50, Epoch [6/50], Loss: 0.0735
Folder 50, Epoch [11/50], Loss: 0.0766
Folder 50, Epoch [16/50], Loss: 0.0517
Folder 50, Epoch [21/50], Loss: 0.0494
Folder 50, Epoch [26/50], Loss: 0.0468
Folder 50, Epoch [31/50], Loss: 0.0463
Folder 50, Epoch [36/50], Loss: 0.0441
Folder 50, Epoch [41/50], Loss: 0.0393
Folder 50, Epoch [46/50], Loss: 0.0359
Average PSNR on test dataset: 11.3808 dB
Folder 75, Epoch [1/50], Loss: 0.1227
Folder 75, Epoch [6/50], Loss: 0.1130
Folder 75, Epoch [11/50], Loss: 0.1002
Folder 75, Epoch [16/50], L